# Get Coordinates #

This script is used to get all the coordinates of each pose's keypoints and saves it into a CSV file.

### Import dependencies ###

In [27]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
import mediapipe as mp
from typing import List, Optional, Tuple, Union, Mapping
from mediapipe.framework.formats import landmark_pb2
from dataclasses import dataclass
import os
import csv

### Resize image function ###

In [8]:
DESIRED_HEIGHT = 800
DESIRED_WIDTH = 800

def resize(image):
    h, w = image.shape[:2]
    if h < w:
        return cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
    else:
        return cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))

### Import images ###

In [9]:
images = {}
images_path = './cropped_pose/'

for image in os.listdir(images_path):
    f = os.path.join(images_path, image)

    if os.path.isfile(f):
        images[f] = (resize(cv2.imread(f)))

### Run mediapipe ###

In [66]:
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles
csv_file = 'pose_coordinates.csv'

if os.path.exists(csv_file):
    os.remove(csv_file)

with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.7, model_complexity=2) as pose:
    with open(csv_file, 'w', encoding='UTF8', newline="") as f:
        writer = csv.writer(f)
        for image in images:
            results = pose.process(cv2.cvtColor(images[image], cv2.COLOR_BGR2RGB))

            image_height, image_width, _ = images[image].shape
            if not results.pose_landmarks:
                continue

            writer.writerow([image, results.pose_landmarks])

### Mediapipe Constants and Tools ###

In [11]:
WHITE_COLOR = (224, 224, 224)
BLACK_COLOR = (0, 0, 0)
RED_COLOR = (0, 0, 255)

_PRESENCE_THRESHOLD = 0.05
_VISIBILITY_THRESHOLD = 0.05
_BGR_CHANNELS = 3

@dataclass
class DrawingSpec:
    color: Tuple[int, int, int] = WHITE_COLOR
    thickness: int = 2
    circle_radius: int = 2

def _normalize_color(color):
    return tuple(v / 255. for v in color)

def _normalized_to_pixel_coordinates(
        normalized_x: float, normalized_y: float, image_width: int,
        image_height: int) -> Union[None, Tuple[int, int]]:
    """Converts normalized value pair to pixel coordinates."""

    # Checks if the float value is between 0 and 1.
    def is_valid_normalized_value(value: float) -> bool:
        return (value > 0 or math.isclose(0, value)) and (value < 1 or math.isclose(1, value))
        
    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    return x_px, y_px

### Draw landmarks ###

In [12]:
def draw_landmarks(
    image: np.ndarray,
    landmark_list: landmark_pb2.NormalizedLandmarkList,
    connections: Optional[List[Tuple[int, int]]] = None,
    landmark_drawing_spec: Union[DrawingSpec,
                                 Mapping[int, DrawingSpec]] = DrawingSpec(
                                     color=RED_COLOR),
    connection_drawing_spec: Union[DrawingSpec,
                                   Mapping[Tuple[int, int],
                                           DrawingSpec]] = DrawingSpec()):
    """Draws the landmarks and the connections on the image.

    Args:
      image: A three channel BGR image represented as numpy ndarray.
      landmark_list: A normalized landmark list proto message to be annotated on
        the image.
      connections: A list of landmark index tuples that specifies how landmarks to
        be connected in the drawing.
      landmark_drawing_spec: Either a DrawingSpec object or a mapping from hand
        landmarks to the DrawingSpecs that specifies the landmarks' drawing
        settings such as color, line thickness, and circle radius. If this
        argument is explicitly set to None, no landmarks will be drawn.
      connection_drawing_spec: Either a DrawingSpec object or a mapping from hand
        connections to the DrawingSpecs that specifies the connections' drawing
        settings such as color and line thickness. If this argument is explicitly
        set to None, no landmark connections will be drawn.

    Raises:
      ValueError: If one of the followings:
        a) If the input image is not three channel BGR.
        b) If any connetions contain invalid landmark index.
    """
    if not landmark_list:
        return
    if image.shape[2] != _BGR_CHANNELS:
        raise ValueError('Input image must contain three channel bgr data.')
    image_rows, image_cols, _ = image.shape
    idx_to_coordinates = {}
    for idx, landmark in enumerate(landmark_list.landmark):
        if ((landmark.HasField('visibility') and
             landmark.visibility < _VISIBILITY_THRESHOLD) or
            (landmark.HasField('presence') and
             landmark.presence < _PRESENCE_THRESHOLD)):
            continue
        landmark_px = _normalized_to_pixel_coordinates(landmark.x, landmark.y,
                                                       image_cols, image_rows)
        if landmark_px:
            idx_to_coordinates[idx] = landmark_px
    if connections:
        num_landmarks = len(landmark_list.landmark)
        # Draws the connections if the start and end landmarks are both visible.
        for connection in connections:
            start_idx = connection[0]
            end_idx = connection[1]
            if not (0 <= start_idx < num_landmarks and 0 <= end_idx < num_landmarks):
                raise ValueError(f'Landmark index is out of range. Invalid connection '
                                 f'from landmark #{start_idx} to landmark #{end_idx}.')
            if start_idx in idx_to_coordinates and end_idx in idx_to_coordinates:
                drawing_spec = connection_drawing_spec[connection] if isinstance(
                    connection_drawing_spec, Mapping) else connection_drawing_spec
                cv2.line(image, idx_to_coordinates[start_idx],
                         idx_to_coordinates[end_idx], drawing_spec.color,
                         drawing_spec.thickness)
    # Draws landmark points after finishing the connection lines, which is
    # aesthetically better.
    if landmark_drawing_spec:
        for idx, landmark_px in idx_to_coordinates.items():
            drawing_spec = landmark_drawing_spec[idx] if isinstance(
                landmark_drawing_spec, Mapping) else landmark_drawing_spec
            # White circle border
            circle_border_radius = max(drawing_spec.circle_radius + 1,
                                       int(drawing_spec.circle_radius * 1.2))
            cv2.circle(image, landmark_px, circle_border_radius, WHITE_COLOR,
                       drawing_spec.thickness)
            # Fill color into the circle
            cv2.circle(image, landmark_px, drawing_spec.circle_radius,
                       drawing_spec.color, drawing_spec.thickness)

In [68]:
with open(csv_file, 'r') as f:
    reader = csv.reader(f, delimiter=',', skipinitialspace=True)
    for name, data in reader:
        print(name, data)

./cropped_pose/a_hidari_gedan_barai.png landmark {
  x: 0.3913237452507019
  y: 0.17700499296188354
  z: 0.043442994356155396
  visibility: 0.9995575547218323
}
landmark {
  x: 0.391274631023407
  y: 0.1555335819721222
  z: -0.0027227129321545362
  visibility: 0.9998001456260681
}
landmark {
  x: 0.3919898569583893
  y: 0.15513715147972107
  z: -0.0030000582337379456
  visibility: 0.9997807145118713
}
landmark {
  x: 0.3923543095588684
  y: 0.15484100580215454
  z: -0.002771149855107069
  visibility: 0.9997877478599548
}
landmark {
  x: 0.39763256907463074
  y: 0.15389296412467957
  z: 0.03925305977463722
  visibility: 0.9998594522476196
}
landmark {
  x: 0.40376150608062744
  y: 0.15246060490608215
  z: 0.03866018354892731
  visibility: 0.9998899698257446
}
landmark {
  x: 0.4098389148712158
  y: 0.15062978863716125
  z: 0.038284119218587875
  visibility: 0.9999169111251831
}
landmark {
  x: 0.415940523147583
  y: 0.15621605515480042
  z: -0.13368181884288788
  visibility: 0.999745786

In [50]:
fig = plt.figure(figsize=(10, 7))
rows = 5
cols = 5
num = 0

with open(csv_file, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for name, data in reader:
        annotated_image = cv2.imread(name)
        draw_landmarks(
            annotated_image,
            data,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        fig.add_subplot(rows, cols, num+1)
        plt.imshow(annotated_image)
        plt.axis('off')
        plt.title(name.split('/')[2])

AttributeError: 'str' object has no attribute 'landmark'

<Figure size 1000x700 with 0 Axes>

### Plot landmarks ###

In [ ]:
def plot_landmarks(landmark_list: landmark_pb2.NormalizedLandmarkList,
                   connections: Optional[List[Tuple[int, int]]] = None,
                   landmark_drawing_spec: DrawingSpec = DrawingSpec(
                       color=RED_COLOR, thickness=5),
                   connection_drawing_spec: DrawingSpec = DrawingSpec(
                       color=BLACK_COLOR, thickness=5),
                   elevation: int = 10,
                   azimuth: int = 10):

    """Plot the landmarks and the connections in matplotlib 3d.

    Args:
      landmark_list: A normalized landmark list proto message to be plotted.
      connections: A list of landmark index tuples that specifies how landmarks to
        be connected.
      landmark_drawing_spec: A DrawingSpec object that specifies the landmarks'
        drawing settings such as color and line thickness.
      connection_drawing_spec: A DrawingSpec object that specifies the
        connections' drawing settings such as color and line thickness.
      elevation: The elevation from which to view the plot.
      azimuth: the azimuth angle to rotate the plot.

    Raises:
      ValueError: If any connection contains an invalid landmark index.
    """
    
    if not landmark_list:
        return
    plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    ax.view_init(elev=elevation, azim=azimuth)
    plotted_landmarks = {}
    for idx, landmark in enumerate(landmark_list.landmark):
        if ((landmark.HasField('visibility') and
             landmark.visibility < _VISIBILITY_THRESHOLD) or
            (landmark.HasField('presence') and
             landmark.presence < _PRESENCE_THRESHOLD)):
            continue
        ax.scatter3D(
            xs=[-landmark.z],
            ys=[landmark.x],
            zs=[-landmark.y],
            color=_normalize_color(landmark_drawing_spec.color[::-1]),
            linewidth=landmark_drawing_spec.thickness)
        plotted_landmarks[idx] = (-landmark.z, landmark.x, -landmark.y)
    if connections:
        num_landmarks = len(landmark_list.landmark)
        # Draws the connections if the start and end landmarks are both visible.
        for connection in connections:
            start_idx = connection[0]
            end_idx = connection[1]
            if not (0 <= start_idx < num_landmarks and 0 <= end_idx < num_landmarks):
                raise ValueError(f'Landmark index is out of range. Invalid connection '
                                 f'from landmark #{start_idx} to landmark #{end_idx}.')
            if start_idx in plotted_landmarks and end_idx in plotted_landmarks:
                landmark_pair = [
                    plotted_landmarks[start_idx], plotted_landmarks[end_idx]
                ]
                ax.plot3D(
                    xs=[landmark_pair[0][0], landmark_pair[1][0]],
                    ys=[landmark_pair[0][1], landmark_pair[1][1]],
                    zs=[landmark_pair[0][2], landmark_pair[1][2]],
                    color=_normalize_color(
                        connection_drawing_spec.color[::-1]),
                    linewidth=connection_drawing_spec.thickness)
    plt.show()

In [ ]:
plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS, elevation=10, azimuth=0)